## Filtering the Dataframe properly
1. order msut sum to 45
2. MatchID must be a valid number
3. Lane must be between 1 = 3
4. Hero ID must be between 1 - 138
5. MatchID must be unique and sum to exactly 10

In [152]:
import pandas as pd
df = pd.read_csv('large_amounts_of_data.csv')

# Keep only the first 10 consecutive rows of each match ID to prevent duplicated data
df_cleaned = df.groupby('MatchID').head(10)

# put the cleaned up data into a new file
df_cleaned.reset_index(drop=True, inplace=True)
df_cleaned.to_csv('cleaned_data.csv', index=False)

df = pd.read_csv("cleaned_data.csv")
# Added in the bracer key late so the early models dont have it but it just means they were 0 so we can just make those values go to zero
df["bracer"].fillna(0, inplace=True)

#1. the ordrr must sum to 45
grouped = df.groupby("MatchID")["Order"].sum()
filtered_match_ids = grouped[grouped == 45].index
# Create a new DataFrame with only the rows corresponding to the filtered "MatchID"
df = df[df["MatchID"].isin(filtered_match_ids)]

# 2 and 3 remove the values of the dataframe where the match id is invalid or the lane is biggerr than 3 as this is ambigous and bad data 
cleaned_df = df[(df['MatchID'] > 100000) & (df["Lane"] < 4) & (df["Lane"] > 0)]

#4
cleaned_df = cleaned_df[cleaned_df["Hero ID"].between(1, 138)]

# 5. A valid match id must have exactly 10 matches
matchID_counts = cleaned_df['MatchID'].value_counts()
valid_matchID = matchID_counts[matchID_counts == 10].index
final_df = cleaned_df[cleaned_df["MatchID"].isin(valid_matchID)]
final_df


,Hero ID,Team,Order,MatchID,Lane,smoke_of_deceit,boots,flask,blood_grenade,clarity,...,ring_of_regen,sobi_mask,null_talisman,buckler,headdress,ring_of_basilius,wind_lace,boots_of_elves,dust,bracer
10,13,1,9,7425319964,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
11,46,1,6,7425319964,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
12,138,1,2,7425319964,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
13,83,1,5,7425319964,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0.0
14,77,1,0,7425319964,3,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109961,14,0,3,6619455028,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
109962,114,0,5,6619455028,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
109963,39,0,8,6619455028,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
109964,19,0,0,6619455028,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0


## Sanity checks on the Data 

In [144]:
#confirmation check that we cleaned properly
result = final_df.groupby("MatchID")["Order"].sum()
result = result.reset_index()
filter = result[result["Order"] != 45]
filter

final_df["MatchID"].value_counts().sort_values(ascending=True)


MatchID
7425319964    10
7424434469    10
7424448081    10
7425165105    10
7425127995    10
              ..
6751431258    10
6751534727    10
6751648250    10
6751109412    10
6751834824    10
Name: count, Length: 10040, dtype: int64

## ConvolutionNeural Network for 2D matrix

In [28]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Create a Sequential model
model = keras.Sequential()

# Add a 2D convolutional layer with 32 filters
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(10, 44, 1)))

# Add a max-pooling layer
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the output from the previous layer
model.add(layers.Flatten())

# Add a dense (fully connected) layer with 64 units
model.add(layers.Dense(64, activation='relu'))

# Add the output layer with as many units as the number of lane classes
# Adjust the number of units according to your specific problem
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


2023-11-09 01:50:49.584716: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 01:50:49.798404: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-09 01:50:49.798444: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-09 01:50:49.798851: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-09 01:50:49.855901: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-09 01:50:49.867583: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 42, 32)         320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 4, 21, 32)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 2688)              0         
                                                                 
 dense (Dense)               (None, 64)                172096    
                                                                 
 dense_1 (Dense)             (None, 10)                650       
                                                                 
Total params: 173066 (676.04 KB)
Trainable params: 173066 (676.04 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Assuming df is your DataFrame

# Prepare your input data (features)
X = df.iloc[:, 5:].values  # Select columns from 6th to the end (excluding "Lane")
X = X.reshape(-1, 10, 44, 1)  # Reshape to (number_of_samples, 10, 44, 1)

# Prepare your target data (labels)
y = df["Lane"].values.reshape(-1, 1)  # Assuming "Lane" is the target variable

# Perform one-hot encoding on the labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and compile the 2D CNN model (as defined in the previous answer)

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy * 100:.2f}%')
